In [1]:
!date

Fri May 19 06:28:01 EDT 2023


In [2]:
pwd

'/mmfs1/data/aglinska/BC-actionpred-seg'

In [3]:
from matplotlib import pyplot as plt
from PIL import Image, ImageEnhance,ImageStat
import skvideo
import skvideo.io
import numpy as np
import os
from tqdm import tqdm
import cv2

from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import pandas as pd
import random

In [4]:
#fnt_dir = './FreeMono.ttf'
fnt_dir = './Arial Bold.ttf'
fnt = ImageFont.truetype(fnt_dir, 40)

In [5]:
def load_video(fn):
    #fn = 'ID4_h1_1.mp4'
    videodata = skvideo.io.vread(fn)
    return videodata

In [6]:
def get_text_im(text='000',nframes=150):
    img = Image.new('RGB', (640,360))
    d = ImageDraw.Draw(img)
    d.text((320-len(text)*10, 150), text, fill=(255, 255, 255),font=fnt)
    return np.array([np.array(img) for _ in range(nframes)])

In [7]:
def save_video(videodata,ofn='outvideo.mp4'):
    #skvideo.io.vwrite(ofn, videodata.astype(np.uint8),outputdict={'-pix_fmt' : 'yuv420p','-codec': 'h264', '-filter:v' : 'fps=20'})
    skvideo.io.vwrite(ofn, videodata.astype(np.uint8))

In [96]:
def safe_mkdir(fn):
    import os
    if not os.path.exists(fn):
        os.mkdir(fn)

### Define paths

In [135]:

indir_root = './Data/pytorch-data/xl_121_15_action_data_bg/test/'
#vid_fn_temp = './Data/haa500_v1_1_yolo_seg4/{this_cat}/{this_cat}_{vid_num}.mp4'
#vid_fn_temp = './Data/haa500_v1_1_yolo_seg4/{this_cat}/{this_cat}_{vid_num}_item_0_inp_seg.mp4'
vid_fn_temp = './Data/haa500_v1_1_yolo_seg4/{this_cat}/{this_cat}_{vid_num}_item_0_inv_seg.mp4'

ofdir = './Data/01_gifs_for_mTurk_bg/'
safe_mkdir(ofdir)

### Generate legend.CSV

In [136]:
plt.ioff()

In [137]:
mturk_categories = np.load('mturk_categories.npy')
n = len(mturk_categories)

In [138]:
codes = [f'{np.random.randint(low=0,high=999):03d}' for _ in range(n)]

In [139]:
df = pd.DataFrame()

In [140]:
# Fix names for some categories
mturk_categories_nice = ['skateboard forward',
 'golf part',
 'pull ups',
 'tight-rope walking',
 'using lawn mower',
 'chainsaw tree',
 'closing door',
 'using scythe',
 'draw handgun',
 'yoga gate',
 'play maracas',
 'play panpipe',
 'using typewriter',
 'balloon animal',
 'add new car tire',
 'screw car tire',
 'play grandpiano',
 'shake cocktail',
 'arm wave',
 'wear face mask',
 'play gong',
 'firemaking with bow',
 'gym plank',
 'play saxophone',
 'play sanxian',
 'softball pitch',
 'floss dance',
 'basketball jabstep',
 'pottery wheel',
 'watering plants',
 'volleyball set',
 'yoga updog',
 'whipping',
 'yoga firefly',
 'styling hair',
 'cutting onion',
 'swinging axe on a tree',
 'weightlifting stand',
 'play serpent',
 'squash backhand',
 'dog highfive',
 'play ukulele',
 'smoking inhale',
 'shuffle dance',
 'taekwondo high block',
 'play hulusi',
 'bench dip',
 'climb stair',
 'play sheng',
 'playing cajon drum',
 'play tambourine',
 'play cornett',
 'play  piccolo',
 'shoveling snow',
 'handshake dog',
 'handsaw',
 'rock balancing',
 'play recorder',
 'weightlifting overhead',
 'running in place',
 'jack up car',
 'yoga locust',
 'jump rope',
 'fish-hunting pull',
 'yoga pigeon',
 'neck side pull stretch',
 'pouring wine',
 'fish-hunting hold',
 'play melodic',
 'chalkboard',
 'whistle two hands',
 'gym squat',
 'play leaf-flute',
 'blowing balloon',
 'play ocarina',
 'adjusting glasses',
 'eat apple',
 'play erhu',
 'play triangle',
 'blow gun']

mturk_categories_nice = np.array(mturk_categories_nice)

In [141]:
df['action'] = list(mturk_categories_nice)
df['file'] = [f'gif_{i:03d}.gif' for i in range(80)]
df['code'] = codes

In [142]:
#[i.replace('_',' ') for i in mturk_categories]

In [143]:
answer_choices = []
for c in range(n):
    answer_choices.append( [random.choice(list(mturk_categories_nice[np.arange(0,n)!=c])) for _ in range(5)] )

In [144]:
answer_choices = np.array(answer_choices)
answer_choices.shape

(80, 5)

In [145]:
for c in range(n):
    answer_choices[c,np.random.randint(0,5)] = mturk_categories_nice[c]

In [146]:
df['choice_1'] = list(answer_choices[:,0])
df['choice_2'] = list(answer_choices[:,1])
df['choice_3'] = list(answer_choices[:,2])
df['choice_4'] = list(answer_choices[:,3])
df['choice_5'] = list(answer_choices[:,4])

In [147]:
df.to_csv(os.path.join(ofdir,'data_legend.csv'))

In [148]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [149]:
df.head(5)

,action,file,code,choice_1,choice_2,choice_3,choice_4,choice_5
0,skateboard forward,gif_000.gif,802,watering plants,play panpipe,taekwondo high block,play sheng,skateboard forward
1,golf part,gif_001.gif,214,golf part,volleyball set,play gong,handshake dog,adjusting glasses
2,pull ups,gif_002.gif,280,using lawn mower,fish-hunting hold,pull ups,whipping,play cornett
3,tight-rope walking,gif_003.gif,614,tight-rope walking,floss dance,yoga locust,yoga updog,styling hair
4,using lawn mower,gif_004.gif,229,play grandpiano,shake cocktail,play hulusi,play triangle,using lawn mower


### Make videos .gifs

In [150]:
for c in tqdm(range(n)):
    this_cat = mturk_categories[c]
    #indir = f'./Data/pytorch-data/xl_121_15_action_data_orig/test/{this_cat}'
    #indir = f'./Data/pytorch-data/xl_121_15_action_data_seg/test/{this_cat}'
    #indir = f'./Data/pytorch-data/xl_121_15_action_data_orig/test/{this_cat}'
    indir = os.path.join(indir_root,this_cat)
    frames = [frame for frame in os.listdir(indir) if not frame.startswith('.')]
    frames.sort()
    frame = frames[0]
    vid_num = frame.split('_')[0]
    #vid_fn = f'./Data/haa500_v1_1_yolo_seg4/{this_cat}/{this_cat}_{vid_num}.mp4'
    vid_fn = vid_fn_temp.format(this_cat=this_cat,vid_num=vid_num)
    video_data = load_video(vid_fn)
    video_data = video_data[0:min(video_data.shape[0],150),:,:,:]
    #video_data.shape
    
    arr = np.concatenate((
                get_text_im(text='5',nframes=25),
                get_text_im(text='4',nframes=25),
                get_text_im(text='3',nframes=25),
                get_text_im(text='2',nframes=25),
                get_text_im(text='1',nframes=25),
                video_data,
                get_text_im(text=codes[c],nframes=250)),axis=0)
    
    save_video(arr,os.path.join(ofdir,f'gif_{c:03d}.gif'))
    
    plt.figure(figsize=(20,10))
    plt.subplot(1,3,1)
    plt.imshow(arr[0,:,:,:])
    plt.axis('off')
    plt.subplot(1,3,2)
    plt.imshow(arr[126,:,:,:])
    plt.axis('off')
    plt.subplot(1,3,3)
    plt.imshow(arr[-1,:,:,:])
    plt.axis('off')
    plt.suptitle(mturk_categories[c],fontsize=20,y=0.66)
    plt.savefig(os.path.join(ofdir,f'im_{c:03d}.jpg'))
    plt.close()

100%|██████████| 80/80 [03:10<00:00,  2.38s/it]


In [114]:
pwd

'/mmfs1/data/aglinska/BC-actionpred-seg'

In [9]:
#!zip -r './Data/01_gifs_for_mTurk_bg.zip' './Data/01_gifs_for_mTurk_bg/'